In [0]:
!pip install openai==0.28

In [0]:
dbutils.library.restartPython()

In [0]:
import openai
openai.api_type = "azure"
openai.api_base = "https://oai-dk.openai.azure.com/"
openai.api_version = "2023-12-01-preview"
openai.api_key = dbutils.secrets.get('akvdesafiokinea','azure-oai-dk')

In [0]:
#Função para Gerar Restrição
def gerar_restricao(regra,dic):
    message_text = [
                {"role":"system",
                "content":
                    """
                    Você receberá o texto em relação a uma regra e um dicionário de valores em torno daquela regra. Sua resposta deve ser uma função de restrição no estilo scipy no formato:
                    def restricao(x):
                        return x
                    e o tipo de restrição "ineq" para inequações e "eq" para equações
                """
                },
                {"role":"user","content":f"Texto para a regra: {regra}, JSON com Resultados: {dic}"}
               ]
    
    completion = openai.ChatCompletion.create(
        engine="gpt35turbo16k",
        messages = message_text,
        temperature=0,
        max_tokens=1200,
        top_p=0.80,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None)

    return completion.to_dict()['choices'][0]['message']['content']

In [0]:
ex_dic= {'ativo': 'UNEG11', 'pior_rating_permitido': 2, 'rating do ativo': 6, 'posicao_ativo': 118787475.0} #A partir do Parser
ex_regra = "Vedação - Ativos de Crédito Privado c/ Rating Diferente de A" #A partir de nxenq_regras.Descricao

resposta = gerar_restricao(ex_regra,ex_dic)
print(resposta)
